In [30]:
print("hello world")

hello world


In [31]:

import csv

val = []
test = []
neg, pos = 0, 0
with open("../dataset/original/aggre_fact_sota.csv", "r") as f:
    csvreader = list(csv.reader(f))
    csvreader = csvreader[1:]
    # print(len(csvreader))
    for row in csvreader:
        dataset = row[0]
        origin = row[1]
        cut = row[7]
        label = row[6]

        if dataset not in ["CLIFF", "Goyal21"]:
            continue
        if origin == "cnndm":
            continue


        if cut == "val":
            val.append(row)
        else:
            test.append(row)
            if label == "1":
                pos += 1
            else:
                neg += 1

print(len(val), len(test), neg, pos)     


200 200 111 89


In [32]:
def acc_distinguish_threshold(cuts: list, th: float) -> float:
    correct = 0
    for row in cuts:
        dae_score = float(row[8])
        label = int(row[6])
        if dae_score >= th:
            correct += 1 if label == 1 else 0
        else:
            correct += 1 if label == 0 else 0
    return correct / len(cuts)    

In [33]:
from sklearn.metrics import balanced_accuracy_score

def bacc_distinguish_threshold(cuts: list, th: float) -> float:
    y_true = [int(row[6]) for row in cuts]
    y_pred = [1 if float(row[8]) >= th else 0 for row in cuts]
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    return balanced_acc

In [34]:
def acc_decide_threshold(val: list) -> float:
    ths = [i / 1000 for i in range(1001)]
    best_th = 0
    max_score = 0
    for th in ths:
        score = acc_distinguish_threshold(val, th)
        # print(th, score)
        if score > max_score:
            max_score = score
            best_th = th
    return best_th

In [47]:
def bacc_decide_threshold(val: list) -> float:
    ths = [i / 1000 for i in range(1001)]
    best_th = 0
    max_score = 0
    for th in ths:
        score = bacc_distinguish_threshold(val, th)
        # print(th, score, max_score, best_th)
        if score > max_score:
            max_score = score
            best_th = th
    return best_th

In [36]:
def acc_calc_score(val: list, test: list) -> float:
    th = acc_decide_threshold(val)
    score = acc_distinguish_threshold(test, th)
    return score

In [49]:
def bacc_calc_score(val: list, test: list) -> float:
    th = bacc_decide_threshold(val)
    print(th)
    score = bacc_distinguish_threshold(test, th)
    return score

In [50]:
print(acc_calc_score(val, test), bacc_calc_score(val, test))

0.0 0.5 0 0
0.001 0.5 0.5 0.0
0.002 0.5080645161290323 0.5 0.0
0.003 0.5080645161290323 0.5080645161290323 0.002
0.004 0.5080645161290323 0.5080645161290323 0.002
0.005 0.5161290322580645 0.5080645161290323 0.002
0.006 0.5241935483870968 0.5161290322580645 0.005
0.007 0.5362903225806451 0.5241935483870968 0.006
0.008 0.5377758913412564 0.5362903225806451 0.007
0.009 0.5311969439728353 0.5377758913412564 0.008
0.01 0.5246179966044142 0.5377758913412564 0.008
0.011 0.5246179966044142 0.5377758913412564 0.008
0.012 0.5246179966044142 0.5377758913412564 0.008
0.013 0.5326825127334465 0.5377758913412564 0.008
0.014 0.5341680814940577 0.5377758913412564 0.008
0.015 0.5462648556876061 0.5377758913412564 0.008
0.016 0.5462648556876061 0.5462648556876061 0.015
0.017 0.5502971137521222 0.5462648556876061 0.015
0.018 0.5543293718166383 0.5502971137521222 0.017
0.019 0.5517826825127334 0.5543293718166383 0.018
0.02 0.5558149405772496 0.5543293718166383 0.018
0.021 0.5638794567062818 0.555814940577

In [39]:
from sklearn.metrics import confusion_matrix

th = acc_decide_threshold(val)
y_true = [int(row[6]) for row in test]
y_pred = [1 if float(row[8]) >= th else 0 for row in test]

cm = confusion_matrix(y_true, y_pred)

print(cm)

[[89 22]
 [33 56]]


In [40]:
from sklearn.metrics import confusion_matrix

th = bacc_decide_threshold(val)
y_true = [int(row[6]) for row in test]
y_pred = [1 if float(row[8]) >= th else 0 for row in test]

cm = confusion_matrix(y_true, y_pred)

print(cm)

[[89 22]
 [33 56]]
